In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))

path_fig = '/home/demitau/current_owncloud/output/memerr/laptop_home'
os.environ['FIG_MEMORY_ERRORS_STAB_AND_STOCH'] = path_fig

import gc

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl

In [ ]:
#d = '/home/demitau/current_owncloud/merr_data/travel_beh_an/ccpurebeh'
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
#fnb = 'dima_test_context_change_20230309_164851'
#fnb = '2023-SE1-pilot2_context_change_20230310_161928'
fnb = '2023-SE1-017_context_change_20230425_161049'
fn = fnb + '.log'
fnp = fnb + '.param'

In [ ]:
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
from datetime import datetime 
fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

load = True
#load = True
if load:
    #dfallphs = pd.read_pickle(pjoin(d,'row=frame_allph.pkl.zip'))
    dfc_all      = pd.read_pickle(pjoin(d,'row=frame.pkl.zip'))
    dfcc_all     = pd.read_pickle(pjoin(d,'row=trial.pkl.zip'))
    dfcpc_all    = pd.read_pickle(pjoin(d,'row=pause.pkl.zip'))
    dfctc_all    = pd.read_pickle(pjoin(d,'row=target_chagne.pkl.zip'))

coln_pairs = [ ('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang'),
        ('error_area2_signed_nn_scaled_ed', None,
               'error_area2_signed_nn_scaled_ed_nonhit',
                'signed_area2_nn_scaled_ed') ] 


fnf = pjoin(d,'es_row=trial.pkl.zip')
dt = datetime.fromtimestamp(os.stat(fnf).st_mtime )
print(dt)

df_all_multi_tsz_orig = pd.read_pickle(fnf)
#assert set( df_all_multi_tsz_orig['error_data_for_calc'] ) == set(  [cp[-1] for cp in coln_pairs])

subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[3]

### recalc ES

In [ ]:
assert 'error' not in dfcc_all.columns
#df_all = dfcc_all.drop(labels=['index', 'level_0'],axis=1)  #.copy()
#df_all['non_hit_not_adj'] = df_all['nonhit']
dfcc_escalc = dfcc_all.copy()
dfcc_escalc['target_inds']     = dfcc_all['tgti_to_show']
dfcc_escalc['trials']          = dfcc_all['trial_index']
dfcc_escalc['environment'] = 1111

# for err sens calc
#df_all['error']          = df_all['error_area_signed']
##############################
subjects_ = subjects
print(subjects_)

q = 0.04
a = dfcc_escalc['error_lh_ang'].abs()
thr = a.quantile(q); print(thr)
dfcc_escalc['error_lh_ang_nonhit'] = a > thr

a = dfcc_escalc['error_area2_signed_nn_scaled_ed'].abs()
thr = a.quantile(q); print(thr)
dfcc_escalc['error_area2_signed_nn_scaled_ed_nonhit'] = a > thr

In [ ]:
from behav_proc import computeErrSensVersions
envs_cur = ['all']
tgt_inds_cur =[None]

block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ None ]
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'
#time_lockeds=['target', 'feedback', 'trial_end']
time_lockeds=['target', 'trial_end']

# coln_pairs = [ ('error_area2_signed_nn_scaled_ed',
#                 None,
#                'error_area2_signed_nn_scaled_ed_nonhit',
#                 'signed_area2_nn_scaled_ed') ]
coln_pairs = []
coln_pairs += [('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

from base2 import areaBetweenTraj, calcNormCoefSectorArea
norm_coef = calcNormCoefSectorArea(params)
def trajPair2corr(dftraj1, dftraj2):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params ) * norm_coef

dfouts = []
import sys, traceback
try:
    for coln_error,coln_correction_calc,coln_nh, calc_name  in coln_pairs:
        for time_locked in time_lockeds:
            df_all_multi_tsz, ndf2vn = computeErrSensVersions(dfcc_escalc, envs_cur,block_names_cur,
                pertvals_cur,gseqcs_cur,tgt_inds_cur,
                dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
                subj_list = subjects_, error_type=error_type,
                trial_shift_sizes = [1],
                addvars=[], target_info_type = 'locs',                                        
                computation_ver = 'computeErrSens3', 
                coln_nh = coln_nh,
                time_locked = time_locked,
                coln_nh_out = coln_nh + '_shifted',
                coln_error =coln_error,                                                                              
                coln_correction_calc = coln_correction_calc,                
                trajPair2corr = trajPair2corr,
                df_fulltraj = dfc_all,
                verbose=1,)
            df_all_multi_tsz['error_data_for_calc'] = calc_name
            dfouts += [df_all_multi_tsz]
    df_all_multi_tsz = pd.concat(dfouts).reset_index()
    
except Exception as e:
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]
    psf = ei
    
    lfprev = None
    lfs = []
    lf = None
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame
        display(stackframe)
        lfprev = lf
        lf = stackframe.f_locals
        lfs += [lf]
        psf = psf.tb_next    
        if psf is None:
            break

    print('EXC',e)

if coln_correction_calc is not None:
    df_all_multi_tsz['coln_corrc'] = df_all_multi_tsz[coln_correction_calc]
df_all_multi_tsz['coln_error'] = df_all_multi_tsz[coln_error]

bigEStis = df_all_multi_tsz.query('err_sens > 10')['trial_index'].to_numpy()
display(bigEStis)
bigEStis_neg = df_all_multi_tsz.query('err_sens < -10')['trial_index'].to_numpy()
#display(bigEStis_neg)

In [ ]:
import gc; gc.collect()

# Compare truncations

In [ ]:
edfc = 'lh_ang', 'signed_area2_nn_scaled_ed'

In [ ]:
import pingouin as pg
numtrain = params['num_training']
from behav_proc import (qs_notspec, qs_notspec_not_afterspec,  
                        spectrials, qs_easy0)


#qs_to_check = qs_notspec
qs_to_check = qs_notspec + ' and not ' + qs_easy0

## lh_ang

In [ ]:
df_all_multi_tsz = df_all_multi_tsz_orig

cols = ['subject', 'trial_index', 'error_data_for_calc',
       'trial_shift_size', 'trial_group_col_calc', 'time_locked']
df_all_multi_tsz.duplicated(cols).any()

In [ ]:
edfc = 'lh_ang'
tl = 'target'
#tl = 'trial_end'
#f'subject == "{subj}"' 
qs_es = ('error_data_for_calc == @edfc'
          ' and trial_shift_size == 1 '
          ' and trial_group_col_calc == "trials"'
          ' and time_locked == @tl'
         )  


#dfes = df_all_multi_tsz_orig.query(qs_es)
dfes = df_all_multi_tsz.query(qs_es)
#dfes

In [ ]:
dfes.groupby(['subject','trial_index']).size().max()

In [ ]:
qs_to_check = qs_notspec + ' and not ' + qs_easy0
cc = 3 * np.pi / 180 
qs_to_check += ' and prev_error.abs() >= @cc'

In [ ]:
from behav_proc import getTruncDep
ttrs = getTruncDep(dfes, qs_to_check)
gc.collect()

In [ ]:
print( ttrs['qs'].values[0] )
cols = ['alternative',  'truncaprs',  'p-val', 
        'power','dof', 'low_mean', 'hi_mean']
ttrs.loc[ttrs["p-val"] <= 0.05, cols]

In [ ]:
#sns.catplot?

In [ ]:
truncapd

In [ ]:
sns.scatter(data=dfes, y='err_sens')

In [ ]:
# super slow
#sns.histplot(data=dfes, y='err_sens')

In [ ]:
sns.vio

In [ ]:
params['dist_tgt_from_home']

In [ ]:
#dfes['error_lh_ang'].max()

In [ ]:
for truncapd in pds[:1]:
    print(truncapd, qs_to_check)
#    truncapd = pds[0]
    dfes_goog, locs = truncateDf( dfes.query(qs_to_check), 'err_sens', **truncapd, infnan_handling='discard',
            verbose=0, retloc=1 )
plt.hist(dfes_goog['err_sens'], bins=90);
plt.gca().axvline(x=0, c='r', ls=':')

plt.figure()
plt.hist(dfes_goog['err_sens'], bins=200, range=(-10,10));
plt.gca().axvline(x=0, c='r', ls=':')
#plt.xlim(-400,400)

In [ ]:
#sns.histplot?

In [ ]:
print(truncapd)

In [ ]:
plt.hist(dfes_goog['err_sens'], bins=200);

In [ ]:
# TODO: thr 1 deg

In [ ]:
plt.hist?

In [ ]:
plt.hist(dfes_goog['err_sens'], bins=200, range=(-10,10));
plt.gca().axvline(x=0, c='r', ls=':')
#plt.xlim(-400,400)

In [ ]:
for truncapd in pds[:1]:
#    truncapd = pds[0]
    dfes_goog, locs = truncateDf( dfes, 'err_sens', **truncapd, infnan_handling='discard',
            verbose=0, retloc=1 )
    ylim = (-100,100)
    dfes_goog_ = dfes_goog.query('err_sens >= @ylim[0] and '
                               'err_sens <= @ylim[1]')
    fg = sns.catplot(kind='violin', data=dfes_goog_, y='err_sens')
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.set_ylim(*ylim)
        ax.set_title(str(truncapd))

In [ ]:
for truncapd in pds[:1]:
#    truncapd = pds[0]
    dfes_goog, locs = truncateDf( dfes, 'err_sens', **truncapd, infnan_handling='discard',
                                     verbose=0, retloc=1 )
    fg = sns.catplot(kind='violin', data=dfes_goog, y='err_sens')
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        #ax.set_ylim(-5,5)
        ax.set_title(str(truncapd))

In [ ]:
truncapd = pds[0]
dfes_goog, locs = truncateDf( dfes, 'err_sens', **truncapd, infnan_handling='discard',
                                 verbose=1, retloc=1 )
fg = sns.catplot(kind='violin', data=dfes_goog, y='err_sens', col='subject', col_wrap=1)
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':')
    ax.set_ylim(-5,5)

In [ ]:
subj

## area

In [ ]:
df_all_multi_tsz['error_data_for_calc'].unique()

In [ ]:
edfc = 'signed_area2_nn_scaled_ed'
tl = 'target'
#tl = 'trial_end'
#f'subject == "{subj}"' 
qs_es = ('error_data_for_calc == @edfc'
          ' and trial_shift_size == 1 '
          ' and trial_group_col_calc == "trials"'
          ' and time_locked == @tl'
         )  


#dfes = df_all_multi_tsz_orig.query(qs_es)
dfes = df_all_multi_tsz.query(qs_es)
#dfes

In [ ]:
# dependence on truncation params
from behav_proc import truncateDf
print( qs_to_check )

pds = [dict(q=None), dict(q=None, low=-50, hi=50),dict(q=None, low=-30, hi=30), dict(q=None, low=-10, hi=10),
       dict(q=None, low=-1, hi=1),
    dict(q=0.05),dict(q=0.02), dict(q=None),
      dict(q=0.05, trunc_hi = 0),dict(q=0.02, trunc_hi =0), dict(q=None, trunc_hi =0),
      dict(q=0.05, trunc_low = 0),dict(q=0.02, trunc_low =0), dict(q=None, trunc_low =0),
      dict(q=0.05, trunc_low = 0, abs=True),dict(q=0.02, trunc_low =0, abs=True), dict(q=None, trunc_low =0, abs=True)]
rs = []
for truncapd in pds:
    print('   truncapd=',truncapd)
    dfes_goog, locs = truncateDf( dfes, 'err_sens', **truncapd, 
                        infnan_handling='discard',
                         verbose=1, retloc=1 )
    for alt in ['greater','less','two-sided']:
        r = pg.ttest(dfes_goog.query(qs_to_check)['err_sens'], 0 , alternative=alt)
        r['truncaprs'] = repr(truncapd)
        r['hi_mean'] = None
        r['low_mean'] = None
        low = locs.get('qlow',None)
        hi = locs.get('qhi',None)
        if low is not None:
            r['low_mean'] = low.mean()
        else:
            r['low_mean'] = locs.get('low',None)
        if hi is not None:
            r['hi_mean'] = hi.mean()
        else:
            r['hi_mean'] = locs.get('hi',None)
            
        r['qs'] = qs_to_check
        rs += [r]
    
    del dfes_goog
    #break
    #display(pg.ttest(dfes_goog.query(qs_notspec)['err_sens'], 0 , alternative='less'))
ttrs = pd.concat(rs)
gc.collect()

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import ttest_1samp

In [ ]:
ttest_1samp

In [ ]:
a = ttest_1samp(dfes_goog['err_sens'], 0, alternative='less')
b = ttest_1samp(dfes_goog['err_sens'], 0, alternative='greater')
c = ttest_1samp(dfes_goog['err_sens'], 0, alternative='two-sided')
display(a,b,c)

In [ ]:
print( ttrs['qs'].values[0] )
cols = ['alternative',  'truncaprs',  'p-val', 
        'power','dof', 'low_mean', 'hi_mean']
ttrs.loc[ttrs["p-val"] <= 0.05, cols]

In [ ]:
for truncapd in pds[:1]:
#    truncapd = pds[0]
    print(truncapd)
    dfes_goog, locs = truncateDf( dfes, 'err_sens', **truncapd, infnan_handling='discard',
            verbose=0, retloc=1 )

In [ ]:
plt.hist(dfes_goog['err_sens'], bins=200, range=(-10,10));
plt.gca().axvline(x=0, c='r', ls=':')
#plt.xlim(-400,400)

In [ ]:
plt.hist(dfes_goog['err_sens'], bins=200);#, range=(-10,10));
plt.gca().axvline(x=0, c='r', ls=':')
#plt.xlim(-400,400)

In [ ]:
for truncapd in pds[:1]:
    print(truncapd, qs_to_check)
#    truncapd = pds[0]
    dfes_goog, locs = truncateDf( dfes.query(qs_to_check), 'err_sens', **truncapd, infnan_handling='discard',
            verbose=0, retloc=1 )
plt.hist(dfes_goog['err_sens'], bins=90);
plt.gca().axvline(x=0, c='r', ls=':')

plt.figure()
plt.hist(dfes_goog['err_sens'], bins=200, range=(-10,10));
plt.gca().axvline(x=0, c='r', ls=':')
#plt.xlim(-400,400)